In [1]:
import os

print(os.getcwd())
os.chdir('/mnt/cpp_projects/low-level-deep-learning/')
print(os.getcwd())

/mnt/cpp_projects/low-level-deep-learning/examples
/mnt/cpp_projects/low-level-deep-learning


In [2]:
from sklearn.datasets import load_diabetes
import numpy as np

In [3]:
diabetes = load_diabetes()

In [4]:
data = diabetes.data.astype(np.float32)
targets = diabetes.target.astype(np.float32)
features = diabetes.feature_names

print(data.shape, targets.shape)

(442, 10) (442,)


In [5]:
# First add a dimension to targets
targets_ = targets.reshape(-1, 1)
print(targets_.shape)

(442, 1)


A note on examing the shapes of the backward pass as it is more complicated to understand

In [6]:
# examining the data transformations of the forward pass
W = np.random.randn(data.shape[1], 1)
print(data.shape, W.shape, type(data))
(data @ W).shape
N = data @ W
P = N + 3.2
print("N:", N.shape, "P:", P.shape)

# examining the data transformations of the gradients
dLdP = -2 * (targets_ - P)
dPdN = np.ones_like(N)
dPdB = np.ones((1, 1), dtype=np.float32)

dLdN = dLdP * dPdN

dNdW = data.transpose()
dLdW = dNdW @ dLdN

dLdB = (dLdP * dPdB)
print("pre aggregated dLdB", dLdB.shape, "post aggregated", dLdB.sum(axis=0).shape)
print("dLdP:", dLdP.shape, "dLdN:", dLdN.shape, "dLdW:", dLdW.shape)



(442, 10) (10, 1) <class 'numpy.ndarray'>
N: (442, 1) P: (442, 1)
pre aggregated dLdB (442, 1) post aggregated (1,)
dLdP: (442, 1) dLdN: (442, 1) dLdW: (10, 1)


In [7]:
import cppapi
import random

W = np.random.randn(data.shape[1], 1).astype(np.float32)
random_intercept = random.random()
linear = cppapi.LinearRegression(data.shape[0], data.shape[1], random_intercept, W, 1)
loss = linear._forward_lin_reg_one_step(data, targets_, cppapi.Loss.RMSE)

In [8]:
print("cpp api loss", loss)

cpp api loss 169.66909790039062


In [9]:
from python_tests.manual_linear_regression import forward_linear_regression, loss_gradients
from typing import Dict

weights: Dict[str, np.ndarray] = {}
weights['W'] = W
weights['B'] = np.array([[random_intercept]], dtype=np.float32)
pyloss, forward_info = forward_linear_regression(data, targets_, weights)

In [10]:
import matplotlib.pyplot as plt


print(forward_info.keys())

print("Python/C++ N output close?", np.allclose(forward_info['N'], linear.N))
print("Python/C++ N output close?", np.allclose(forward_info['P'], linear.P))
print(forward_info['P'].shape, linear.P.shape)
print(type(float(pyloss)), type(loss))
print("Loss discrepency Py/C++", pyloss, "/", loss)

dict_keys(['X', 'N', 'P', 'y'])
Python/C++ N output close? True
Python/C++ N output close? True
(442, 1) (442, 1)
<class 'float'> <class 'float'>
Loss discrepency Py/C++ 169.66913 / 169.66909790039062


In [11]:
# perform the python backward differentiation step
loss_gradients_output = loss_gradients(forward_info, weights)
# perform the C++ API backward differentiation step
linear._gradient_backward_step(data, targets_)

In [12]:
print("Weights (dLdW) Python==C++:", np.allclose(linear.dLdW, loss_gradients_output['W']))
print(loss_gradients_output['B'], linear.dLdB)

Weights (dLdW) Python==C++: True
[-133664.66] -133664.640625


In [13]:
linear.train(data, targets_, 1000, 100, cppapi.Loss.RMSE, 1e-3)


Iteration: [0], loss: [7.69]
Iteration: [1], loss: [5.63]
Iteration: [2], loss: [4.90]
Iteration: [3], loss: [4.93]
Iteration: [4], loss: [4.84]
Iteration: [5], loss: [4.81]
Iteration: [6], loss: [4.74]
Iteration: [7], loss: [4.84]
Iteration: [8], loss: [4.78]
Iteration: [9], loss: [4.69]
Iteration: [10], loss: [4.61]
Iteration: [11], loss: [4.76]
Iteration: [12], loss: [4.79]
Iteration: [13], loss: [4.58]
Iteration: [14], loss: [4.60]
Iteration: [15], loss: [4.78]
Iteration: [16], loss: [4.80]
Iteration: [17], loss: [4.76]
Iteration: [18], loss: [4.60]
Iteration: [19], loss: [4.65]
Iteration: [20], loss: [4.66]
Iteration: [21], loss: [4.87]
Iteration: [22], loss: [4.58]
Iteration: [23], loss: [4.56]
Iteration: [24], loss: [4.72]
Iteration: [25], loss: [4.59]
Iteration: [26], loss: [4.59]
Iteration: [27], loss: [4.59]
Iteration: [28], loss: [4.58]
Iteration: [29], loss: [4.50]
Iteration: [30], loss: [4.59]
Iteration: [31], loss: [4.54]
Iteration: [32], loss: [4.49]
Iteration: [33], lo